In [1]:
from pathlib import Path
import shutil
from bids import BIDSLayout
import pandas as pd
import numpy as np
from io import StringIO
import re
import nibabel as nb
from subprocess import run
from collections import namedtuple
import json
import shutil

# from office365.runtime.auth.authentication_context import AuthenticationContext
# from office365.sharepoint.client_context import ClientContext
# from office365.sharepoint.file import File 
# import getpass
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.width', 1000)


from joblib import parallel_backend, Parallel, delayed
from IPython.core.display import display, HTML
display(HTML("<style>"
    + "#notebook { padding-top:0px !important; } " 
    + ".container { width:100% !important; } "
    + ".end_space { min-height:0px !important; } "
    + "</style>"))

from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
project_root = Path('/data/MBDU/midla')
bids_dir = project_root / 'data/bids'
#new_bids_dir = Path('/data/MBDU/bids/mid_longitudinal_20190910')
derivatives_dir = project_root / 'data/derivatives'
swarm_cmd_dir = project_root / 'swarm/fmriprep/swarm_cmds'
sing_img_dir = Path('/data/MBDU/singularity_images/')
#image_path = (sing_img_dir/'fmriprep_1.4.1.simg').as_posix()
#fs_licence_path = sing_img_dir/'license.txt'
#assert fs_licence_path.exists()
#fs_licence_path = fs_licence_path.as_posix()

n_dummy = 4
outlier_fraction_limit = 0.1
fd_limit = 0.8
n_jobs = 20

In [3]:
# change this for other fmriprep runs
run_name = 'fmriprepv20.1.0_20200528_2mm_clifix_noaroma'
rest_run_name = 'fmriprepv20.1.0_20200528_2mm_clifix_noaroma'
fmriprep_out = derivatives_dir / 'fmriprep' / run_name
fmriprep_out_rest = derivatives_dir / 'fmriprep' / rest_run_name
#cmd_file = swarm_cmd_dir / run_name
#swarm_log_dir = project_root / ('swarm/fmriprep/swarm_logs' + run_name)


In [4]:
assert (fmriprep_out / 'dataset_description.json').exists()
database_file='/data/MBDU/midla/notebooks/pybids_fmriprepv20.1.0_20200528_2mm_clifix_noaroma'
%time layout = BIDSLayout(bids_dir, database_file=database_file, derivatives=fmriprep_out.as_posix())

CPU times: user 24.6 ms, sys: 4.69 ms, total: 29.3 ms
Wall time: 135 ms


In [5]:
subjects = layout.get_subjects()

In [6]:
dir_path = re.compile("dir-(?P<direction>[a-z]+)")

In [7]:
def get_dims(img_path):
    img_shape = nb.load(img_path.as_posix()).shape
    if len(img_shape) == 3:
        img_shape = list(img_shape) + [1]
    return img_shape

In [8]:
group = []
for sid in subjects:
    subj_out_dir = fmriprep_out / f'sub-{sid}'
    row = {}
    row['subject'] = sid
    row['outpath'] = subj_out_dir
    # Check to see if there's any output at all
    row['contents'] = len(list(subj_out_dir.glob('*'))) > 0
    row['fig_contents'] = len(list((subj_out_dir /f'out/fmriprep/sub-{sid}/figures/').glob('*'))) > 0
    # check for any crash files
    crash_files = sorted(list(subj_out_dir.glob('**/crash*')))
    row['crash'] = len(crash_files) > 0
    if len(crash_files) > 0:
        row['crash_files'] = crash_files
        nodes = []
        for crash in crash_files:
            cl = crash.read_text().split('\n')
            nodes.append(cl[0].replace("Node:", ""))
        row['crash_nodes'] = nodes
    group.append(row)
group = pd.DataFrame(group)

In [9]:
# Would normally be 0, but in this case I know these don't have output
#assert len(group.query('~contents')) == 0
group['report_path'] = group.outpath.apply(lambda x: list(x.glob('out/fmriprep/sub*.html')))
group.loc[group.report_path.apply(lambda x: x == []), 'report_path'] = np.nan
group.loc[group.report_path.notnull(), 'report_path'] = group.loc[group.report_path.notnull(), 'report_path'].str[0]

# Making a bids dataframe takes too long, going to do it with get


In [10]:
%time func_scans = layout.get(return_type='file', scope='raw', extension='nii.gz', task='mid', datatype='func')
%time rest_scans = layout.get(return_type='file', scope='raw', extension='nii.gz', task='rest', datatype='func')
%time anat_scans = layout.get(return_type='file', scope='raw', extension='nii.gz', datatype='anat')

CPU times: user 31.3 s, sys: 668 ms, total: 32 s
Wall time: 32 s
CPU times: user 2min 2s, sys: 5.29 s, total: 2min 7s
Wall time: 2min 8s
CPU times: user 1min 18s, sys: 4.55 s, total: 1min 23s
Wall time: 1min 23s


In [11]:
%time func_scans = layout.get(return_type='file', scope='raw', extension='nii.gz', task='mid', datatype='func')

CPU times: user 31.2 s, sys: 841 ms, total: 32.1 s
Wall time: 32.1 s


In [12]:
bids_dat = []
# Rest is still screwed up
#for ff in rest_scans + func_scans + anat_scans:

for ff in func_scans + anat_scans:
    bd = layout.parse_file_entities(ff)
    bd['path'] = ff
    bids_dat.append(bd)
bids_dat = pd.DataFrame(bids_dat)

In [13]:
bids_qcr = bids_dat.merge(group.loc[:, ['subject', 'report_path']], how='left', on='subject')
anat_rows = bids_qcr.query('datatype == "anat" & suffix == "T1w"').groupby('subject').first().reset_index()
anat_rows['session'] = 'ALL'
anat_rows['order'] = 0
anat_rows['on_report'] = True

In [14]:
hpat = re.compile('<div id="(?P<result>datatype-func_desc-summary[A-Za-z\-_0-9]+)"')
sespat = re.compile('_session-(?P<res>[A-Z0-9a-z]+)_')
taskpat = re.compile('_task-(?P<res>[A-Z0-9a-z]+)')
runpat = re.compile('_run-(?P<res>[A-Z0-9a-z]+)')

func_orders = []
for ix, df in bids_qcr.groupby('report_path'):
    break
    func_report = []
    i = 1
    report = ix.read_text()
    res = hpat.findall(report)
    for rr in res:
        fr = {}
        fr['subject'] = df.subject.unique()[0]
        fr['session'] = sespat.search(rr).group('res')
        fr['task'] = taskpat.search(rr).group('res')
        try:
            fr['run'] = int(runpat.search(rr).group('res'))
        except AttributeError:
            pass
        fr['order'] = i
        i += 1
        func_report.append(fr)
    func_report = pd.DataFrame(func_report)
    func_orders.append(func_report)

In [15]:
func_report = []
i = 1
report = ix.read_text()
res = hpat.findall(report)

In [16]:
func_rows = bids_qcr.query('datatype != "anat"')

# Try making a single section group report

In [17]:
html_head = r"""<?xml version="1.0" encoding="utf-8" ?>
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="en" lang="en">
<head>
<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
<meta name="generator" content="Docutils 0.12: http://docutils.sourceforge.net/" />
<title></title>
<script src="https://code.jquery.com/jquery-3.3.1.slim.min.js" integrity="sha384-q8i/X+965DzO0rT7abK41JStQIAqVgRVzpbzo5smXKp4YfRvH+8abtTE1Pi6jizo" crossorigin="anonymous"></script>
<script src="https://stackpath.bootstrapcdn.com/bootstrap/4.1.3/js/bootstrap.min.js" integrity="sha384-ChfqqxuZUCnJSK3+MXmPNIyE6ZbWh2IMqE241rYiqJxyMiZ6OW/JmZQ5stwEULTy" crossorigin="anonymous"></script>
<script type="text/javascript">
  var subjs = []
  function updateCounts(){
    var counts = {report:{"-1":0, "1":0, "0":0}}

    subjs.forEach(function(val, idx, arr){
      counts.report[val.report] += 1;
    })

    $("#nrpass").text(counts.report["1"])
    $("#nrfail").text(counts.report["0"])
    $("#nrtodo").text(counts.report["-1"])
  }

  function qc_update(run_id, stage, value) {
    if (stage == 'report') {
      subjs[run_id][stage] = parseInt(value)
      updateCounts();
    }
    else {
      subjs[run_id][stage] = value
    }
  }
  
  function update_all(stage, value) {
    subjs.forEach( subj => {subj[stage]=value})
  }

  function get_csv(items) {
    // https://stackoverflow.com/questions/44396943/generate-a-csv-file-from-a-javascript-array-of-objects
    let csv = ''

    // Loop the array of objects
    for(let row = 0; row < items.length; row++){
        let keysAmount = Object.keys(items[row]).length
        let keysCounter = 0

        // If this is the first row, generate the headings
        if(row === 0){

           // Loop each property of the object
           for(let key in items[row]){

              // This is to not add a comma at the last cell
              // The '\r\n' adds a new line
              csv += key + (keysCounter+1 < keysAmount ? ',' : '\r\n' )
              keysCounter++
           }
           let keysCounterb = 0
           for(let key in items[row]){
               csv += items[row][key] + (keysCounterb+1 < keysAmount ? ',' : '\r\n' )
               keysCounterb++
           }
        }else{
           for(let key in items[row]){
               csv += items[row][key] + (keysCounter+1 < keysAmount ? ',' : '\r\n' )
               keysCounter++
           }
        }

        keysCounter = 0
    }

    // Once we are done looping, download the .csv by creating a link
    let link = document.createElement('a')
    link.id = 'download-csv'
    link.setAttribute('href', 'data:text/plain;charset=utf-8,' + encodeURIComponent(csv));
    link.setAttribute('download', 'manual_qc.csv');
    document.body.appendChild(link)
    document.querySelector('#download-csv').click()
  }
  
  function parse_id(idstr) {
    return idstr.split('_')[0].split('-')[1]
  }

  var observer = new IntersectionObserver(function(entries, observer) {
      entries.forEach(entry => {
        eid = parse_id(entry.target.id)
        if (entry['intersectionRatio'] == 1 && subjs[eid]['been_on_screen'] == false) {
          subjs[eid]['been_on_screen'] = true
        }
        else if (entry['intersectionRatio'] == 0 && subjs[eid]['been_on_screen'] == true && subjs[eid]['report'] == -1) {
          subjs[eid]['report'] = 1
          observer.unobserve(entry.target)
          updateCounts();
          radioid = 'inlineRadio' + eid
          document.querySelectorAll('[name=' + radioid + ']')[0].checked = true
        }
        /* Here's where we deal with every intersection */
      });
    }
  , {root:document.querySelector('#scrollArea'), threshold:[0,1]});

 </script>
<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.1.3/css/bootstrap.min.css" integrity="sha384-MCw98/SFnGE8fJT3GXwEOngsV7Zt27NXFoaoApmYm81iuXoPkFOJwJ8ERdknLPMO" crossorigin="anonymous">
<style type="text/css">
.sub-report-title {}
.run-title {}

h1 { padding-top: 35px; }
h2 { padding-top: 20px; }
h3 { padding-top: 15px; }

.elem-desc {}
.elem-caption {
    margin-top: 15px
    margin-bottom: 0;
}
.elem-filename {}

div.elem-image {
  width: 100%;
  page-break-before:always;
}

.elem-image object.svg-reportlet {
    width: 100%;
    padding-bottom: 5px;
}
body {
    padding: 65px 10px 10px;
}

.boiler-html {
    font-family: "Bitstream Charter", "Georgia", Times;
    margin: 20px 25px;
    padding: 10px;
    background-color: #F8F9FA;
}

div#boilerplate pre {
    margin: 20px 25px;
    padding: 10px;
    background-color: #F8F9FA;
}

</style>
</head>
<body>"""

html_foot = """<script type="text/javascript">
    function toggle(id) {
        var element = document.getElementById(id);
        if(element.style.display == 'block')
            element.style.display = 'none';
        else
            element.style.display = 'block';
    }

</script>
<script>

updateCounts();
document.querySelectorAll('[id^="id"]').forEach(img => {observer.observe(img)})

</script>
</body>
</html>"""

In [18]:
nav_head = """<nav class="navbar fixed-top navbar-expand-lg navbar-light bg-light">
<div class="collapse navbar-collapse">
    <ul class="navbar-nav">
        <li class="nav-item dropdown">
            <a class="nav-link dropdown-toggle" id="navbarFunctional" data-toggle="dropdown" aria-haspopup="true" aria-expanded="false" href="#">Jump to Report</a>
            <div class="dropdown-menu" aria-labelledby="navbarFunctional">"""
nav_foot = """            </div>
        </li>
      </ul>
    </div>

        <div class="navbar-header">
           Ratings: <span id="nrpass" class="badge badge-success">0</span> <span id="nrfail" class="badge badge-danger">0</span> <span id="nrtodo" class="badge badge-warning">0</span>
         </div>
         <div class="navbar-text">
           <button type="button" class="btn btn-info btn-sm" id="csv_download" onclick="get_csv(subjs)">Download CSV</button>

         </div>
</div>
</nav>"""

In [39]:
def make_nav_entry(idx, subject,datatype,suffix, session=None, run=None, task=None, desc=None, space=None):
    ses_href = ''
    run_href = ''
    task_href = ''
    desc_href = ''
    space_href = ''
    ses_label = ''
    run_label = ''
    task_label = ''
    desc_label = ''
    space_label = ''
    if session is not None:
        ses_href = f'_session-{session}'
        ses_label = f'session <span class="bids-entity">{session}</span>'
    if run is not None:
        run_href = f'_run-{run}'
        run_label = f', run <span class="bids-entity">{run}</span>'
    if task is not None:
        task_href = f'_task-{task}'
        task_label = f', task <span class="bids-entity">{task}</span>'
    if desc is not None:
        desc_href = f'_desc-{desc}'
    if space is not None:
        space_href = f'_space-{space}'
    nav_entry=f"""<a class="dropdown-item" href="#id-{idx}_subject-{subject}_datatype-{datatype}{desc_href}{run_href}{ses_href}{space_href}_suffix-{suffix}{task_href}">Subject <span class="bids-entity">{subject}</span>{ses_label}{run_label}{task_label}.</a>"""
    return nav_entry

In [20]:
def make_brain_mask_blurb(subject, idx, chunk, session, run, task, echo=None, img_path=None):
    echo_str = ''
    if pd.notnull(echo):
        echo_str =f'_echo-{echo}'
    if img_path is None:
        img_path = f'../sub-{subject}/out/fmriprep/sub-{subject}/figures'
    blurb= f"""
    <div id="id-{idx}_subject-{subject}_datatype-func_desc-rois_run-{run}_session-{session}_suffix-bold_task-{task}">
      <script type="text/javascript">
        var subj_qc = {{id:{idx},chunk:{chunk}, report_type: "func_rois", subject: "{subject}", session: "{session}", task: "{task}", run: "{run}",  report:null, been_on_screen:false}}
      </script>
      <h2> subject <span class="bids-entity">{subject}</span>, session <span class="bids-entity">{session}</span>, run <span class="bids-entity">{run}</span> , task <span class="bids-entity">{task}</span> </h2>
      <div class="radio">
        <label><input type="radio" name="inlineRadio{idx}" id="inlineRating1" value="1" onclick="qc_update({idx}, 'report', this.value)"> Good </label>
        <label><input type="radio" name="inlineRadio{idx}" id="inlineRating0" value="0" onclick="qc_update({idx}, 'report', this.value)"> Bad</label>
      </div>
      <p> Notes: <input type="text" id="box{idx}" oninput="qc_update({idx}, 'note', this.value)"></p>
<img class="svg-reportlet" src="{img_path}/sub-{subject}_ses-{session}_task-{task}_run-{run}{echo_str}_desc-rois_bold.svg" style="width: 100%" />
</div>
  <script type="text/javascript">
    subj_qc["report"] = -1
    subjs.push(subj_qc)
  </script>"""
    return blurb

def make_anat_reg_blurb(subject, idx, chunk, img_path=None):
    if img_path is None:
        img_path = f'../sub-{subject}/out/fmriprep/sub-{subject}/figures'
    blurb= f"""
    <div id="id-{idx}_subject-{subject}_datatype-anat_space-MNI152NLin2009cAsym_suffix-T1w">
      <script type="text/javascript">
        var subj_qc = {{id:{idx},chunk:{chunk}, report_type: "anat_reg", subject: "{subject}", session: null, task:null, run:null, report:null, been_on_screen:false}}
      </script>
    <h2> subject <span class="bids-entity">{subject}</span></h2>
    <div class="radio">
      <label><input type="radio" name="inlineRadio{idx}" id="inlineRating1" value="1" onclick="qc_update({idx}, 'report', this.value)"> Good </label>
      <label><input type="radio" name="inlineRadio{idx}" id="inlineRating0" value="0" onclick="qc_update({idx}, 'report', this.value)"> Bad</label>
    </div>
     <p> Notes: <input type="text" id="box{idx}" oninput="qc_update({idx}, 'note', this.value)"></p>
    <p class="elem-caption">Spatial normalization of the T1w image to the <code>MNI152NLin2009cAsym</code> template.</p>                    <object class="svg-reportlet" type="image/svg+xml" data="{img_path}/sub-{subject}_space-MNI152NLin2009cAsym_T1w.svg">
Problem loading figure sub-{subject}/figures/sub-{subject}_space-MNI152NLin2009cAsym_T1w.svg. If the link below works, please try reloading the report in your browser.</object>
</div>
<div class="elem-filename">
    Get figure file: <a href="{img_path}/sub-{subject}_space-MNI152NLin2009cAsym_T1w.svg" target="_blank">sub-{subject}/figures/sub-{subject}_space-MNI152NLin2009cAsym_T1w.svg</a>
</div>
  <script type="text/javascript">
    subj_qc["report"] = -1
    subjs.push(subj_qc)
  </script>"""
    return blurb

def make_anat_surf_blurb(subject, idx, chunk, img_path=None):
    if img_path is None:
        img_path = f'../sub-{subject}/out/fmriprep/sub-{subject}/figures'
    blurb= f"""
            <div id="id-{idx}_subject-{subject}_datatype-anat_desc-reconall_suffix-T1w">
            <script type="text/javascript">
              var subj_qc = {{id:{idx},chunk:{chunk}, report_type: "anat_surf", subject: "{subject}", session: null, task:null, run:null, report:null, been_on_screen:false}}
            </script>
            <h2> subject <span class="bids-entity">{subject}</span></h2>
                
<h3 class="run-title">Surface reconstruction</h3><p class="elem-caption">Surfaces (white and pial) reconstructed with FreeSurfer (<code>recon-all</code>) overlaid on the participant's T1w template.</p> 
<div class="radio">
      <label><input type="radio" name="inlineRadio{idx}" id="inlineRating1" value="1" onclick="qc_update({idx}, 'report', this.value)"> Good </label>
      <label><input type="radio" name="inlineRadio{idx}" id="inlineRating0" value="0" onclick="qc_update({idx}, 'report', this.value)"> Bad</label>
    </div>
     <p> Notes: <input type="text" id="box{idx}" oninput="qc_update({idx}, 'note', this.value)"></p>

<img class="svg-reportlet" src="{img_path}/sub-{subject}_desc-reconall_T1w.svg" style="width: 100%" />
</div>
<div class="elem-filename">
    Get figure file: <a href="{img_path}/sub-{subject}_desc-reconall_T1w.svg" target="_blank">sub-{subject}/figures/sub-{subject}_desc-reconall_T1w.svg</a>
</div>
<script type="text/javascript">
  subj_qc["report"] = -1
  subjs.push(subj_qc)
</script>"""
    return blurb

In [21]:
func_rows['echo'] = np.nan

/gpfs/gsfs11/users/MBDU/midla/env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [22]:
# Number on each report page
nchunks = 50

In [23]:
clean_funcs = func_rows.loc[(func_rows.echo == '1') | func_rows.echo.isnull()].sort_values([ 'subject','session','task', 'run']).reset_index(drop=True).reset_index().rename(columns={'index':'ind'})
clean_funcs['report_chunk'] = clean_funcs.ind//nchunks

/gpfs/gsfs11/users/MBDU/midla/env/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [24]:
dat_for_anat = clean_funcs.groupby('subject').first().reset_index().drop('ind', axis=1).reset_index().rename(columns={'index':'ind'})

In [25]:

dat_for_anat['report_chunk'] = dat_for_anat.ind//nchunks

In [26]:
single_report_path = fmriprep_out / 'single_item_reports_test2'
single_report_path

PosixPath('/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/single_item_reports_test2')

In [27]:
reviewer_initials = """
 <p> Initials: <input type="text" id="initials_box" oninput="update_all('rater', this.value)"></p>
"""

In [ ]:
anat_reg_n = 0
single_report_path = fmriprep_out / 'single_item_reports_test2'
if not single_report_path.exists():
    single_report_path.mkdir()
local_single_report_path = single_report_path / 'local_reports'
if not local_single_report_path.exists():
    local_single_report_path.mkdir()
local_imgs_path = local_single_report_path / 'imgs'
if not local_imgs_path.exists():
    local_imgs_path.mkdir()

In [40]:
all_lines = []
missing_rows = []
for ii, df in dat_for_anat.groupby('report_chunk'):
    df = df.copy()
    df = df.reset_index(drop=True)
    df = df.reset_index().rename(columns={'index':'idx'})
    nav_entries = '\n'.join(df.apply(lambda row: make_nav_entry(row.idx, row.subject, 'anat', 'T1w', space='MNI152NLin2009cAsym'), axis=1).values)
    nav = '\n'.join([nav_head, nav_entries, nav_foot])
    
    
    local_reportlets = []
    rpt_i = 0
    for ix, row in df.iterrows():
            
        local_img_path = local_single_report_path / f'imgs/sub-{row.subject}_space-MNI152NLin2009cAsym_T1w.svg'
        img_path = fmriprep_out / f'sub-{row.subject}/out/fmriprep/sub-{row.subject}/figures/sub-{row.subject}_space-MNI152NLin2009cAsym_T1w.svg'
        if not img_path.exists():
            missing_rows.append(row)
            continue
        rel_path = f'../../../sub-{row.subject}/out/fmriprep/sub-{row.subject}/figures/sub-{row.subject}_space-MNI152NLin2009cAsym_T1w.svg'
        if not local_img_path.exists():
            local_img_path.symlink_to(rel_path)
        report_line =  make_anat_reg_blurb(row.subject, rpt_i, ii, img_path='./imgs')
        if report_line not in all_lines:
            local_reportlets.append(report_line)
            all_lines.append(report_line)
        rpt_i +=1
            
    reportlets = '\n'.join(local_reportlets)

    anat_reg_n += len(reportlets)
    rpt_text = '\n'.join([html_head,
                          nav,
                          reviewer_initials,
                          reportlets,
                          html_foot])
    rpt_path = single_report_path / f'local_reports/anat_reg_{ii:03d}.html'
    rpt_path.write_text(rpt_text)

In [41]:
anat_surf_n = 0

for ii, df in dat_for_anat.groupby('report_chunk'):
    df = df.copy()
    df = df.reset_index(drop=True)
    df = df.reset_index().rename(columns={'index':'idx'})
    nav_entries = '\n'.join(df.apply(lambda row: make_nav_entry(row.idx, row.subject, 'anat', 'T1w', desc='reconall'), axis=1).values)
    nav = '\n'.join([nav_head, nav_entries, nav_foot])
    
    
    local_reportlets = []
    rpt_i = 0
    for ix, row in df.iterrows():
            
        local_img_path = local_single_report_path / f'imgs/sub-{row.subject}_desc-reconall_T1w.svg'
        img_path = fmriprep_out / f'sub-{row.subject}/out/fmriprep/sub-{row.subject}/figures/sub-{row.subject}_desc-reconall_T1w.svg'
        if not img_path.exists():
            missing_rows.append(row)
            continue
        rel_path = f'../../../sub-{row.subject}/out/fmriprep/sub-{row.subject}/figures/sub-{row.subject}_desc-reconall_T1w.svg'
        if not local_img_path.exists():
            local_img_path.symlink_to(rel_path)
        report_line =  make_anat_surf_blurb(row.subject, rpt_i, ii, img_path='./imgs')
        if report_line not in all_lines:
            local_reportlets.append(report_line)
            all_lines.append(report_line)
            rpt_i +=1
            
    reportlets = '\n'.join(local_reportlets)

    anat_surf_n += len(reportlets)
    rpt_text = '\n'.join([html_head,
                          nav,
                          reviewer_initials,
                          reportlets,
                          html_foot])
    rpt_path = single_report_path / f'local_reports/anat_surf_{ii:03d}.html'
    rpt_path.write_text(rpt_text)

In [42]:
func_rois_n = 0
for ii, df in clean_funcs.groupby('report_chunk'):
    df = df.copy()
    df = df.reset_index(drop=True)
    df = df.reset_index().rename(columns={'index':'idx'})
    nav_entries = '\n'.join(df.apply(lambda row: make_nav_entry(row.idx, row.subject, 'func', 'bold', desc='rois', run=row.run, session=row.session, task=row.task), axis=1).values)
    nav = '\n'.join([nav_head, nav_entries, nav_foot])
    
    local_reportlets = []
    rpt_i = 0
    for ix, row in df.iterrows():
        echo_str = ''
        if pd.notnull(row.echo):
            echo_str =f'_echo-{row.echo}'
        local_img_path = local_single_report_path / f'imgs/sub-{row.subject}_ses-{row.session}_task-{row.task}_run-{row.run}{echo_str}_desc-rois_bold.svg'
        img_path = fmriprep_out / f'sub-{row.subject}/out/fmriprep/sub-{row.subject}/figures/sub-{row.subject}_ses-{row.session}_task-{row.task}_run-{row.run}{echo_str}_desc-rois_bold.svg'
        if not img_path.exists():
            missing_rows.append(row)
        rel_path = f'../../../sub-{row.subject}/out/fmriprep/sub-{row.subject}/figures/sub-{row.subject}_ses-{row.session}_task-{row.task}_run-{row.run}{echo_str}_desc-rois_bold.svg'
        if not local_img_path.exists():
            local_img_path.symlink_to(rel_path)
        report_line =  make_brain_mask_blurb(row.subject, rpt_i, ii,
                                             row.session,
                                             row.run,
                                             row.task,
                                             row.echo,
                                             img_path='./imgs')
        if report_line not in all_lines:
            local_reportlets.append(report_line)
            all_lines.append(report_line)
            rpt_i +=1
        reportlets = '\n'.join(local_reportlets)

    func_rois_n += len(reportlets)
    rpt_text = '\n'.join([html_head,
                          nav,
                          reviewer_initials,
                          reportlets,
                          html_foot])
    rpt_path = single_report_path / f'local_reports/func_rois_{ii:03d}.html'
    rpt_path.write_text(rpt_text)